$\Large{\textbf{Asynchronous Inertial Forward-Backward Splitting for Load Sharing}}$

$\textbf{Goal}$

A collection of $N$ controllable buildings, represented as linear dynamical systems, along with an energy storage 
system, represented as a one-state linear battery model, cooperate in order to track a signal. The signal corresponds
to the deviation of the planned power consumption of the buildings to their real-time consumption in addition to power deviations that are caused by uncontrollable loads. The mix of battery + buildings compensates for the aggregate deviation.  

$\textbf{Problem formulation}$

Track power residual signal for $T$ time instants (15 mins intervals) while operating the buildings and the battery within constraints. 

Can be formulated as

\begin{equation}
\begin{aligned}
&{\text{minimize}} && \sum_{t=0}^T\left(\frac{\alpha}{2}\|p^\mathrm{bess}(t) + \sum_{i=1}^N(p_i^\mathrm{cb}(t)-\hat{p}_i^\mathrm{cb}(t))-r(t)\|^2_2 + \sum_{i=1}^N g^\mathrm{cb}_i(p^{\mathrm{cb}}_i,u_i,x_i,y_i) + g^\mathrm{bess}(p^\mathrm{bess})\right)\enspace,
\end{aligned}
\end{equation}
with variables $p^\mathrm{bess}\in\mathbb{R}^T$ denoting the battery's power (generation or consumption), $p_i^\mathrm{cb}$ the $i^\mathrm{th}$ building's power consumption and $u_i,x_i,y_i$ being variables local to building $i$.






The problem comprises three types of terms:
* Tracking term $\frac{\alpha}{2}\|p^\mathrm{bess}(t) + \sum_{i=1}^N(p_i^\mathrm{cb}(t)-\hat{p}_i^\mathrm{cb}(t))-r(t)\|^2_2$: The signal $r$ is generated from the coordinator and should be tracked by the buildings and the battery, who can vary their consumption within a zone that is centered around their planned consumption (denoted by $p^{\mathrm{cb}}_i$ for building $i$) and defined by their operational constraints.
* Building (CB) $i$'s objective: \begin{equation}
g^\mathrm{cb}_i(p^{\mathrm{cb}}_i,u_i,x_i,y_i) := \Big\{\frac{1}{2}\|y_i(t)-T_i^\mathrm{ref}(t)\|^2_2
                                             \;\mid\;(p^{\mathrm{cb}}_i,u_i,x_i,y_i)\in\mathcal{C}^\mathrm{cb}_i \Big\}\enspace,
\end{equation}
where the zones' temperatures $y_i$ have to remain close to a reference $T_i^\mathrm{ref}$ while respecting operational and comfort constraints encoded in the convex set $\mathcal{C}^\mathrm{cb}_i$.
* Battery (BESS) objective: \begin{equation}
g^\mathrm{bess}(p^\mathrm{bess}) := \Big\{\frac{1}{2}\sum_{t=1}^T\|SOC(t)-SOC^\mathrm{ref}(t)\|^2_2
                                             \;\mid\;p^\mathrm{bess}\in\mathcal{C}^\mathrm{bess} \Big\}\enspace,
\end{equation}
where the battery's state of charge $SOC$ needs to remain close to a reference value $SOC^\mathrm{ref}$ while respecting power and capacity constraints encoded in the convex set $\mathcal{C}^\mathrm{bess}$.

$\textbf{Code}$

The code is organized in modules. The *battery* and *building* modules parse the data that have been generated in MATLAB. The *agentBESS* and *agentCB* modules contain all the information regarding the buildings in the form of *types* and *functions*. These are standalone entities containing private information. 

In the algorithm that follows, a *master* module acts as the coordinator. Its responsibility is to enforce the tracking part of the objective as described above.

Finally, the *pgm* and *async_inert_pgm* modules are the Proximal Gradient Method versions (synchronous and asynchronous) that solve the optimization problem.

The following cells set up the simulation, namely they perform the data parsing and initiate the constructors.

In [12]:
###################################################
## Packages, initial data parsing and set globals #
###################################################

# set up state-space models for two agents
include("building.jl")
using Buildings

include("battery.jl")
using Batteries

include("agentBESS.jl")
using AgentsBESS

include("parseData.jl")
using parseData

include("agentCB.jl")
using AgentsCB

include("master.jl")
using Masters

include("pgm.jl")
using ProximalGradient

include("async_inert_pgm.jl")
using AsynchronousProximalGradient

# general purpose packages
# using Plots
using JuMP
using Gurobi
using ECOS
using Distributions
using JLD
using DataFrames
using BenchmarkTools, Compat

# global variables
const N = 5   # agents
const T = 93  # time horizon
const δ = 0.01  # cost regularizer
const α = 1000.0  # tracking cost regularizer
Tref = 22.0  # reference temperature
Tend = 300.0  # simulation time in seconds

# initialize array of contructors for three (types of) builings and one battery
Building = Array{Buildings.Building}(3)
Battery = Array{Batteries.Battery}(1)

# initialize array of constructors for initial states and disturbance signals
x0 = Array{Any}(3)
w = Array{Any}(3)
ref = Array{Any}(1)

# initialize array of constructors for preconditioners
Γ_BESS = Array{Any}(1)
Γ_CB = Array{Any}(1)

1-element Array{Any,1}:
 #undef

In [13]:
#####################################################
## Parsing of signals and building / battery module #
#####################################################

# parse initial states, disturbance and reference signals and preconditioning matrices
x0, w, ref   = parseData.parseSignals(x0, w, ref)
Γ_BESS, Γ_CB = parseData.parsePreconditioners(Γ_BESS, Γ_CB)

# parse building and battery data
building = Buildings.parseBuildings(Building)
battery  = Batteries.parseBattery(Battery, N, T);


(15,5)


(54,18)


(57,19)



In [14]:
#####################################################
## Aggregator CB constuctors                        # 
#####################################################

# initalize consumptions, state-space, constraints and optimization data structures
consume, ss, con, data = Array{Any}(1,N), Array{Any}(1,N), Array{Any}(1,N), Array{Any}(1,N)
AgentCB = Array{Any}(N)

# construct aggregators
types = convert(Array,readtable(string("data/type", ".dat"), separator = ',', header = false))
for i in 1:N
    consume[i], ss[i], con[i], data[i] =  AgentsCB.setupAggregator(1, building[types[i]], Γ_CB, δ)
    AgentCB[i]   = AgentsCB.Aggregator(1, consume[i], ss[i], con[i], data[i])
end

In [15]:
##############################################
## BESS constructor                          # 
##############################################

# initialize data structure and parameters
data   = Array{Any}(1)
SOC0   = 0.5*battery.SoCMax
SOCref = 0.8*battery.SoCMax
AgentBESS = Array{Any}(1)

# BESS aggregator (currently one battery)
system, data = AgentsBESS.setupBESS(battery, SOC0, SOCref, Γ_BESS)
AgentBESS = AgentsBESS.BESS(system, data)
typeof(AgentBESS)

AgentsBESS.BESS

In [16]:
##############################################
## Master constructor                        # 
##############################################

# dispatch plan corresponds to the baseline consumption
p_CB_hat = zeros(T,N)  # baseline consumption per building
p_CB_tot = zeros(T)    # total baseline consumption
for i in 1:N
    p_CB_hat[:,i] = AgentCB[i].consumption.baseline_electrical
    p_CB_tot = p_CB_tot + p_CB_hat[:,i]
end

# intialize selection matrices
S = Array{Any}(1)
E = Array{Any}(1)
Master = Array{Any}(1)

# constructor for the master
Master = Masters.setupMaster(S, E, ref, N, p_CB_tot)

Masters.Master(5,[1.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 1.0],[1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.0],[-0.2253,-0.13693,-0.21021,-0.08137,-0.10065,-0.024183,-0.022123,0.0049664,-0.15601,-0.12624  …  0.031261,0.0028789,0.036412,0.02905,0.09532,-0.002768,0.023753,0.087993,0.21606,-0.013234],[-0.130526,-0.0426833,-0.115994,0.0127743,-0.00656897,0.0698937,0.0719555,0.0990496,-0.0619228,-0.032148  …  0.124922,0.0965801,0.130181,0.122883,0.189206,0.0911706,0.117757,0.182086,0.310238,0.0809617])

In [17]:
# night and day times
night_times = collect([0:6*4-1; 22*4:24*4-1])
day_times = collect(6*4:22*4-1);

# initialize model
X0 = Array{Any}(N)  # initial state 
W  = Array{Any}(N)  # exogenous inputs

for i in 1:N
    if AgentCB[i].system.Nx == 15
        X0[i] = x0[1]
        W[i] = w[1]
    elseif AgentCB[i].system.Nx == 54
        X0[i] = x0[2]
        W[i] = w[2]
    elseif AgentCB[i].system.Nx == 57
        X0[i] = x0[3]
        W[i] = w[3]
    end
end

$\textbf{Solve the optimization problem centrally}$

The problem is solved centrally using the Gurobi optimizer with JuMP. The optimizers is used for verification of the distributed solution.

In [18]:
##################################################
## Solve one optimal control instance centrally  #
##################################################

t = 0
t0 = time_ns() 

agentModel = Model(solver=GurobiSolver())

@variable(agentModel, U[j=1:N,1:AgentCB[j].system.Nu,1:T])  # activation inputs for heating CBs
@variable(agentModel, X[j=1:N,1:AgentCB[j].system.Nx,1:T])  # internal states for CBs
@variable(agentModel, Y[j=1:N,1:AgentCB[j].system.Ny,1:T])  # temperatures for CBs
@variable(agentModel, p_CB[1:T,1:N])                        # total power consumption for CBs
@variable(agentModel, p_BESS[1:T])                          # total power consumption for BESS

X0  # initial states

# dynamics & equality contraints
for j = 1:N
    for i = 1:AgentCB[j].system.Nx
        @constraint(agentModel, X[j,i,1] == (sum(AgentCB[j].system.A[i,l]*X0[j][l] for l = 1:AgentCB[j].system.Nx) + sum(AgentCB[j].system.Bu[i,l]*U[j,l,1] for l = 1:AgentCB[j].system.Nu) + sum(AgentCB[j].system.Bw[i,l]*W[j][l,1] for l = 1:AgentCB[j].system.Nw) ) )
    end
    for i = 1:AgentCB[j].system.Ny
        @constraint(agentModel, Y[j,i,1] == sum(AgentCB[j].system.C[i,l]*X[j,l,1] for l = 1:AgentCB[j].system.Nx) )
    end
end

for j = 1:N
    for t = 1:T-1
        for i = 1:AgentCB[j].system.Nx
            @constraint(agentModel, X[j,i,t+1] == (sum(AgentCB[j].system.A[i,l]*X[j,l,t] for l = 1:AgentCB[j].system.Nx) + sum(AgentCB[j].system.Bu[i,l]*U[j,l,t+1] for l = 1:AgentCB[j].system.Nu) + sum(AgentCB[j].system.Bw[i,l]*W[j][l,t+1] for l = 1:AgentCB[j].system.Nw) ) )
        end

        for i = 1:AgentCB[j].system.Ny
            @constraint(agentModel, Y[j,i,t+1] == sum(AgentCB[j].system.C[i,l]*X[j,l,t+1] for l = 1:AgentCB[j].system.Nx) )
        end

        temp = 0
        for s = 1:AgentCB[j].system.Nu
            temp = temp + U[j,s,t]
        end
        @constraint(agentModel, p_CB[t,j] == temp/(1000*AgentCB[j].consumption.COP))

    end
    temp = 0
    for s = 1:AgentCB[j].system.Nu
        temp = temp + U[j,s,T]
    end
    @constraint(agentModel, p_CB[T,j] == temp/(1000*AgentCB[j].consumption.COP))
end
                                
# inequality constraints
for j = 1:N
    for t = 1:T
        if (t-1) in night_times
            for i = 1:AgentCB[j].system.Ny
                @constraint(agentModel, Y[j,i,t] >= AgentCB[j].constraints.outputNightMin[i])
                @constraint(agentModel, AgentCB[j].constraints.outputNightMax[i] >= Y[j,i,t])
            end
        elseif (t-1) in day_times
            for i = 1:AgentCB[j].system.Ny
                @constraint(agentModel, Y[j,i,t] >= AgentCB[j].constraints.outputDayMin[i])
                @constraint(agentModel, AgentCB[j].constraints.outputDayMax[i] >= Y[j,i,t])
            end
        end

        for i = 1:AgentCB[j].system.Nu # for all rows do the following
            @constraint(agentModel, 0*AgentCB[j].constraints.inputMin[i] <= U[j,i,t])
            @constraint(agentModel, U[j,i,t] <= AgentCB[j].constraints.inputMax[i])
        end
    end
end
   
v_p_CB = vec(p_CB)  
SE = Master.S*Master.E
M = [eye(T) SE]

for i = 1:T                                
    @constraint(agentModel, p_BESS[i] <= AgentBESS.system.pMax)  
    @constraint(agentModel, p_BESS[i] >= AgentBESS.system.pMin) 
    @constraint(agentModel, AgentBESS.system.AA_bess[i]*SOC0 + dot(AgentBESS.system.BB_bess[i,:],p_BESS) <= AgentBESS.system.SoCMax) 
    @constraint(agentModel, AgentBESS.system.AA_bess[i]*SOC0 + dot(AgentBESS.system.BB_bess[i,:],p_BESS) >= AgentBESS.system.SoCMin)
end
     
obj = 0;                                
obj = dot(p_BESS, 0.5*δ*(AgentBESS.system.BB_bess'*AgentBESS.system.BB_bess)*p_BESS) + δ*dot(p_BESS, (AgentBESS.system.BB_bess'*AgentBESS.data.c_bess)) + 0.5*δ*dot(AgentBESS.data.c_bess, AgentBESS.data.c_bess)   
obj += dot([p_BESS;v_p_CB], 0.5*α*(M'*M)*[p_BESS;v_p_CB]) - α*dot([p_BESS;v_p_CB], (M'*Master.ϵ_unc)) + 0.5*α*dot(Master.ϵ_unc, Master.ϵ_unc)  
                                    
for j = 1:N                                    
    obj += 0.5*δ*vecdot(p_CB[:,j]-AgentCB[j].consumption.baseline_electrical, p_CB[:,j]-AgentCB[j].consumption.baseline_electrical)
    for t = 1:T
        for i = 1:AgentCB[j].system.Ny
            obj += 0.5*δ*(Y[j,i,t]-Tref)*(Y[j,i,t]-Tref) 
        end
    end
end

@objective( agentModel, Min, obj )
solve(agentModel)  # solve optimization
gc()               # run garbage collector
                                
t = (time_ns()-t0)/1e9  # time solution

Academic license - for non-commercial use only
Optimize a model with 59613 rows, 37293 columns and 2008986 nonzeros
Model has 13578 quadratic objective terms
Coefficient statistics:
  Matrix range     [7e-10, 2e+01]
  Objective range  [2e-01, 3e+02]
  QObjective range [1e-06, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-05, 2e+04]
Presolve removed 29669 rows and 651 columns
Presolve time: 0.75s
Presolved: 29944 rows, 36732 columns, 1941834 nonzeros
Presolved model has 13578 quadratic objective terms
Ordering time: 0.64s

Barrier statistics:
 Free vars  : 14796
 AA' NZ     : 2.648e+06
 Factor NZ  : 3.750e+06 (roughly 60 MBytes of memory)
 Factor Ops : 5.169e+08 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.74256722e+15 -2.74301942e+15  1.19e+07 9.25e+05  1.08e+09     2s
   1   3.21936452e+14 -3.22261904e+14  4.06e+06 3.16e+05  4.

38.623334148

In [19]:
# get optimizers
v_p_CB_opt = getvalue(v_p_CB)  # 
p_BESS_opt = getvalue(p_BESS)
p_CB_opt   = getvalue(p_CB)

93×5 Array{Float64,2}:
 0.0155582   0.00188379  0.0104733   0.0104733   0.0155582 
 0.0121758   0.00186016  0.0104032   0.0104032   0.0121758 
 0.00979207  0.00183404  0.00987013  0.00987013  0.00979207
 0.00837487  0.00180503  0.00924747  0.00924747  0.00837487
 0.00716721  0.00171804  0.00737395  0.00737395  0.00716721
 0.0237828   0.00190389  0.0116301   0.0116301   0.0237828 
 0.0119359   0.00182878  0.00998431  0.00998431  0.0119359 
 0.00917755  0.00180043  0.00929151  0.00929151  0.00917755
 0.00889068  0.00176892  0.00843478  0.00843478  0.00889068
 0.0078217   0.00168033  0.00674537  0.00674537  0.0078217 
 0.00959299  0.00176264  0.00837313  0.00837313  0.00959299
 0.0336161   0.00205431  0.0168066   0.0168066   0.0336161 
 0.0170707   0.00193285  0.0124818   0.0124818   0.0170707 
 ⋮                                                         
 0.031967    0.00152941  0.0165186   0.0165186   0.031967  
 0.0318465   0.00153329  0.0164959   0.0164959   0.0318465 
 0.0317358   0.00

In [20]:
#################################################
## Parses agents' optimization problems once    #
#################################################

optimizationModelProxCB = Array{Any}(N)  # initialize N prox minimization problems

for i in 1:N
    optimizationModelProxCB[i] = AgentsCB.setupProxCB(AgentCB[i], Tref, T, X0[i], W[i], night_times, day_times, δ);
end

optimizationModelProxBESS = Array{Any}(1)
optimizationModelProxBESS = AgentsBESS.setupProxBESS(AgentBESS, T, SOC0, δ);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


$\textbf{The Async simulation}$

In order to simulate the asycn implementation we have each agent execute its optimization step several times and consequently 
fit a normal distribution $\mathrm{SolveTime(i)}\sim\mathcal{N}(\mu_i,\sigma_i)$ to the sampled execution times. 
In this way we can estimate the time it will take to perform one local update by drawing a sample from its corresponding distribution.

First initialize with $t_i=\mathrm{SolveTime(i)}$, $t=0$ and some $T_{\mathrm{end}}>t$.

The algorithm reads as follows:
1. $j\leftarrow \mathrm{argmin}_i\;t_i$
2. $t\leftarrow t + t_j$
3. $t_j\leftarrow t + \mathrm{SolveTime(j)}$

Repeat until $t\geq T_{\mathrm{end}}$.


In [11]:
#################################################
## Benchmark proximal minimization problems     #
#################################################

# timings in Julia
bench_small    = BenchmarkTools.Trial           # run trial for small building
bench_med      = BenchmarkTools.Trial           # run trial for medium building
bench_large    = BenchmarkTools.Trial           # run trial for large building
TimeCB_small   = Distributions.Normal{Float64}  # fit normal distribution to solve times of small CB
TimeCB_med     = Distributions.Normal{Float64}  # fit normal distribution to solve times of medium CB
TimeCB_large   = Distributions.Normal{Float64}  # fit normal distribution to solve times of large CB

point = randn(T,1)

# solve a group of proximal optimization problems per type of agent 
# and fit normal distributions to the execution times (in sec)
i = 1
for i in 1:N
    if AgentCB[i].system.Nx == 15
        bench_small = @benchmark AgentsCB.solveProxCB(optimizationModelProxCB[i], point, sparse(AgentCB[i].data.Γ))
        TimeCB_small = fit(Normal, getfield(bench_small, :times)/1e9)
        break;
    end
end

for i in 1:N
    if AgentCB[i].system.Nx == 54
        bench_med = @benchmark AgentsCB.solveProxCB(optimizationModelProxCB[i], point, sparse(AgentCB[i].data.Γ))
        TimeCB_med = fit(Normal, getfield(bench_med, :times)/1e9)
        break;
    end
end

for i in 1:N
    if AgentCB[i].system.Nx == 57
        bench_large = @benchmark AgentsCB.solveProxCB(optimizationModelProxCB[i], point, sparse(AgentCB[i].data.Γ))
        TimeCB_large = fit(Normal, getfield(bench_large, :times)/1e9)
        break;
    end
end

# similarly benchmark BESS solve time (in sec)
point      = randn(T)
bench_BESS = BenchmarkTools.Trial
TimeBESS   = Distributions.Normal{Float64}
bench_BESS = @benchmark AgentsBESS.solveProxBESS(optimizationModelProxBESS, point, sparse(AgentBESS.data.Γ))
TimeBESS   = fit(Normal, getfield(bench_BESS, :times)/1e9)

LoadError: UndefVarError: BenchmarkTools not defined

In [ ]:
#################################################
## Generate execution times for simulation      #
#################################################

execTimesMean = zeros(N+1)  # mean execution time per agent
execTimesVar = zeros(N+1)   # variance of execution time per agent
rndUpdate = zeros(N+1)      # sample solve time normal distribution 

# means and variances of solve time for each type of building
CB_small_μ = TimeCB_small.μ
CB_small_σ = TimeCB_small.σ
CB_medium_μ = TimeCB_med.μ
CB_medium_σ = TimeCB_med.σ
CB_large_μ = TimeCB_large.μ
CB_large_σ = TimeCB_large.σ
BESS_μ = TimeBESS.μ
BESS_σ = TimeBESS.σ

# execution time per building agent; samples are to be used to emulate the async updates
for i in 1:N
    if AgentCB[i].system.Nx == 15
        rndUpdate[i] = CB_small_μ + CB_small_σ*randn()
        execTimesMean[i] = CB_small_μ
        execTimesVar[i] = TimeCB_small.σ
    elseif AgentCB[i].system.Nx == 54
        rndUpdate[i] = CB_medium_μ + CB_medium_σ*randn()
        execTimesMean[i] = CB_medium_μ
        execTimesVar[i] = CB_medium_σ
    elseif AgentCB[i].system.Nx == 57
        rndUpdate[i] = CB_large_μ + CB_large_σ*randn()
        execTimesMean[i] = CB_large_μ
        execTimesVar[i] = CB_large_σ
    end
end

# execution time for battery agent
rndUpdate[N+1]     = BESS_μ + BESS_σ*randn() 
execTimesMean[N+1] = BESS_μ
execTimesVar[N+1]  = BESS_σ

# Generate next update times for agent
UpdateQueue = Dict{Int64,Float64}(zip(1:N+1, abs(rndUpdate)));

ExecTimesTot  = UpdateQueue
ExecTimesMean = execTimesMean
ExecTimesVar  = execTimesVar

$\textbf{The Synchronous Proximal Gradient Method}$

The problem is first solved using the regular PGM. 

The gradient of the tracking term $\nabla f(p^\mathrm{cb},p^\mathrm{bess})$, where $f(p^\mathrm{cb},p^\mathrm{bess}):=\frac{\alpha}{2}\|p^\mathrm{bess}(t) + \sum_{i=1}^N(p_i^\mathrm{cb}(t)-\hat{p}_i^\mathrm{cb}(t))-r(t)\|^2_2$, is communicated to the agents (CBs and BESS). Each agent subsequently solves a $\textit{proximal minimization problem of the form}$

\begin{align}
&{\text{minimize}} && g_i(z_i) + \frac{1}{2\gamma}\|z_i-y\|_2^2\enspace,
\end{align}
with variable $z=(p^\mathrm{cb},p^\mathrm{bess})$, $\gamma>0$ an admissible stepsize and $y = z_i^k - γ∇_if(z^k)$.

In [ ]:
#########################################################
## Solve one optimal control instance: Synchronous PGM  #
#########################################################

# initialize PGM (call to constructor)
pgm_u, pgm_y, pgm_x, pgm_v, pgm_P_CB, pgm_P_CB_prev, pgm_P_BESS, pgm_P_BESS_prev, pgm_RES = ProximalGradient.setupPGM(AgentCB, N, T)
pgm_init = ProximalGradient.PGM(pgm_u, pgm_y, pgm_x, pgm_v, pgm_P_CB, pgm_P_CB_prev, pgm_P_BESS, pgm_P_BESS_prev, pgm_RES)

# solve
β = 0.0  # heavy-ball acceleration
η = 0.9  # under-relaxation
pgm, T1 = ProximalGradient.AlgoPGM(pgm_init, N, T, Tend, SOC0, ExecTimesTot, optimizationModelProxCB, optimizationModelProxBESS, AgentBESS, AgentCB, Master, p_BESS_opt, v_p_CB_opt, η, β, δ, α);

$\textbf{The Asynchronous Proximal Gradient Method}$

At each global clock count $k$ one agent $i_k$ updates. The update corresponds to the solution of the prox problem

\begin{align}
&{\text{minimize}} && g_{i_k}(z_{i_k}) + \frac{1}{2\gamma}\|z_{i_k}-y_{i_k}\|_2^2\enspace,
\end{align}
where $y_{i_k} = z_{i_k}^k - γ∇_if(z^{i_k}_\mathrm{read})$ and $z^{i_k}_\mathrm{read}$ corresponds to an outdated version of the decision vector $z$, when agent $i_k$ last read it. 

In [ ]:
############################################################################
## Solve one optimal control instance: Coordinate descent asynchronous PGM #
############################################################################

include("async_inert_pgm.jl")
using AsynchronousProximalGradient

# initialize AsInPGM (call to constructor)
async_pgm_u, async_pgm_y, async_pgm_x, async_pgm_v, async_pgm_P_CB, async_pgm_P_CB_prev, async_pgm_P_BESS, async_pgm_P_BESS_prev, async_pgm_RES, async_pgm_P_CB_local, async_pgm_P_BESS_local, async_pgm_P_CB_local_prev, async_pgm_P_BESS_local_prev = AsynchronousProximalGradient.setupAsInPGM(AgentCB, N, T)
async_pgm_init = AsynchronousProximalGradient.AsInPGM(async_pgm_u, async_pgm_y, async_pgm_x, async_pgm_v, async_pgm_P_CB, async_pgm_P_CB_prev, async_pgm_P_BESS, async_pgm_P_BESS_prev, async_pgm_RES, async_pgm_P_CB_local, async_pgm_P_BESS_local, async_pgm_P_CB_local_prev, async_pgm_P_BESS_local_prev)

# reset times
UpdateQueue  = Dict{Int64,Float64}(zip(1:N+1, abs(rndUpdate)))
ExecTimesTot = UpdateQueue

# solve
η = 0.9  # under-relaxation
β = 0.0  # heavy-ball acceleration
as_pgm, T2, NoUpdates2 = AsynchronousProximalGradient.AlgoAsInPGM(true, false, async_pgm_init, N, T, Tend, ExecTimesTot, ExecTimesMean, ExecTimesVar, optimizationModelProxCB, optimizationModelProxBESS, AgentBESS, AgentCB, Master, p_BESS_opt, v_p_CB_opt, η, β, δ, α);

$\textbf{The Asynchronous 'Aggregated' Proximal Gradient Method}$

Similar to the previous algorithm, with the difference that $\textit{all}$ coordinates of the decision vector $z$ are updated at each iteration $k$. For the coordinates $j\neq i_k$, the last-computed values are used to perform the update.

In [ ]:
############################################################################
## Solve one optimal control instance: 'Aggregated' Asynchronous PGM       #
############################################################################

# Initialize AsInPGM (call to constructor)
async_pgm_u, async_pgm_y, async_pgm_x, async_pgm_v, async_pgm_P_CB, async_pgm_P_CB_prev, async_pgm_P_BESS, async_pgm_P_BESS_prev, async_pgm_RES, async_pgm_P_CB_local, async_pgm_P_BESS_local, async_pgm_P_CB_local_prev, async_pgm_P_BESS_local_prev = AsynchronousProximalGradient.setupAsInPGM(AgentCB, N, T)
async_pgm_init = AsynchronousProximalGradient.AsInPGM(async_pgm_u, async_pgm_y, async_pgm_x, async_pgm_v, async_pgm_P_CB, async_pgm_P_CB_prev, async_pgm_P_BESS, async_pgm_P_BESS_prev, async_pgm_RES, async_pgm_P_CB_local, async_pgm_P_BESS_local, async_pgm_P_CB_local_prev, async_pgm_P_BESS_local_prev)

s# solve
η = 0.9  # under-relaxation
β = 0.0  # heavy-ball acceleration
UpdateQueue  = Dict{Int64,Float64}(zip(1:N+1, abs(rndUpdate)))
ExecTimesTot = UpdateQueue
as_agg_pgm, T3, NoUpdates3 = AsynchronousProximalGradient.AlgoAsInPGM(false, false, async_pgm_init, N, T, Tend, ExecTimesTot, ExecTimesMean, ExecTimesVar, optimizationModelProxCB, optimizationModelProxBESS, AgentBESS, AgentCB, Master, p_BESS_opt, v_p_CB_opt, η, β, δ, α);

$\textbf{The Asynchronous Inertial Proximal Gradient Method}$

At each global clock count $k$ one agent $i_k$ updates. The update corresponds to the solution of the prox problem

\begin{align}
&{\text{minimize}} && g_{i_k}(z_{i_k}) + \frac{1}{2\gamma}\|z_{i_k}-y_{i_k}\|_2^2\enspace,
\end{align}
where $y_{i_k} = z_{i_k}^k - γ∇_if(z^{i_k}_\mathrm{read}) + \beta (z^{i_k}_{i_k,\mathrm{write}}-z^{i_k,\mathrm{prev}}_{i,k\mathrm{write}})$ and $\beta\in [0,1)$.

* $z^{i_k}_\mathrm{read}$ corresponds to an outdated version of the decision vector $z$, when agent $i_k$ last received it from the coordinator.
* $z^{i_k}_\mathrm{write}$ corresponds to an outdated version of the decision vector $z$, when agent $i_k$ last updated the coordinator.
* $z^{i_k,\mathrm{prev}}_\mathrm{write}$ corresponds to an outdated version of the decision vector $z$, the second most recent time that agent $i_k$ updated the coordinator.

In [ ]:
############################################################################
## Solve one optimal control instance: Asynchronous Inertial PGM           #
############################################################################

# initialize AsInPGM (call to constructor)
async_pgm_u, async_pgm_y, async_pgm_x, async_pgm_v, async_pgm_P_CB, async_pgm_P_CB_prev, async_pgm_P_BESS, async_pgm_P_BESS_prev, async_pgm_RES, async_pgm_P_CB_local, async_pgm_P_BESS_local, async_pgm_P_CB_local_prev, async_pgm_P_BESS_local_prev = AsynchronousProximalGradient.setupAsInPGM(AgentCB, N, T)
async_pgm_init = AsynchronousProximalGradient.AsInPGM(async_pgm_u, async_pgm_y, async_pgm_x, async_pgm_v, async_pgm_P_CB, async_pgm_P_CB_prev, async_pgm_P_BESS, async_pgm_P_BESS_prev, async_pgm_RES, async_pgm_P_CB_local, async_pgm_P_BESS_local, async_pgm_P_CB_local_prev, async_pgm_P_BESS_local_prev)

# reset times
UpdateQueue  = Dict{Int64,Float64}(zip(1:N+1, abs(rndUpdate)))
ExecTimesTot = UpdateQueue

# solve
η = 0.9  # under-relaxation
β = 0.0  # heavy-ball acceleration
as_agg_in_pgm, T4, NoUpdates4 = AsynchronousProximalGradient.AlgoAsInPGM(false, true, async_pgm_init, N, T, Tend, ExecTimesTot, ExecTimesMean, ExecTimesVar, optimizationModelProxCB, optimizationModelProxBESS, AgentBESS, AgentCB, Master, p_BESS_opt, v_p_CB_opt, η, β, δ, α)

In [ ]:
# compare all outputs
[vec(as_pgm.P_CB) vec(as_agg_pgm.P_CB) vec(as_agg_in_pgm.P_CB) pgm.P_CB v_p_CB_opt]

In [1]:
## Plotting residuals
pyplot()
using LaTeXStrings

t = max(length(T1), max(length(T2), max(length(T3), length(T4))))
t_vals = Vector[T1[2:end], T2[2:end], T3[2:end], T4[2:end]]
y_vals = Vector[pgm.RES[2:end], as_pgm.RES[2:end], as_agg_pgm.RES[2:end], as_agg_in_pgm.RES[2:end]]
labels = ["Sync PGM", "Async PGM", "Async 'Aggregated' PGM", "Async Inertial PGM"]

Plot_N10 = plot(t_vals, y_vals, color=[:black :orange :green :blue], xaxis = ("t [s]", font(12, "Helvetica")), 
                 yaxis = (L"$\|z-z^\ast\| / \|z^\ast\|$"),
                 titlefont = font(12, "Helvetica"),
                 legendfont = font(9, "Helvetica"),
                 linewidth = 2,
#                  leg = false,
                 label = labels',
                 title = ("Convergence to optimizer for N=5"),
                 layout=1)
yaxis!(:log10, font(12, "Helvetica"))

LoadError: UndefVarError: pyplot not defined